#  **Qwen 2.5 7B com raciocínio (1º Tentativa)**

**Authors:** Renato Freitas Bessa, Adonias Caetano de Oliveira, Ariel Soares Teles

**Institution:** UFDPar - PPGBiotec

**Objective:** responde o exames do revalida com llama 3.1 8B por meio de cadeia de raciocínio (1º tentativa).

In [ ]:
from google.colab import drive

# Montar o drive do Google
drive.mount('/content/drive')

Mounted at /content/drive


## **Loading data**

In [ ]:
import pandas as pd
import re
import random

In [ ]:
#sheet_names = ["2017.01", "2017.02", "2020.01", "2020.02", "2021.01", "2022.01", "2022.02", "2023.01", "2023.02", "2024.01"]
sheet_names = ["2023.02", "2024.01"]

#sheet_names = ["2024.01"]

In [ ]:
url = '<public link to access the questions from the 2017.01 to 2024.01 exams>'
file_id = url.split('/')[-2]
read_url='https://drive.google.com/uc?id=' + file_id

# read the data
provas = pd.read_excel(read_url)
provas.head()

,Perguntas-2017.01,Gabarito-2017.01,Perguntas-2017.02,Gabarito-2017.02,Perguntas-2020.01,Gabarito-2020.01,Perguntas-2020.02,Gabarito-2020.02,Perguntas-2021.01,Gabarito-2021.01,Perguntas-2022.01,Gabarito-2022.01,Perguntas-2022.02,Gabarito-2022.02,Perguntas-2023.01,Gabarito-2023.01,Perguntas-2023.02,Gabarito-2023.02,Perguntas-2024.01,Gabarito-2024.01
0,1.\tUma mulher com 23 anos de idade comparece ...,NULA,1.\tUma mulher com 38 anos de idade procurou a...,C,1.\tAo atender uma primigesta com 15 anos de i...,A,"1.\tUma mulher com 43 anos de idade, com 80 kg...",C,"1.\tPrimigesta, de 25 anos de idade, com 34 se...",B,"1.\tUma mulher com 51 anos de idade, em tratam...",NULA,1.\tIMAGEM,C,"1.\tUm paciente com 54 anos, em acompanhamento...",D,"1.\tUma mulher de 60 anos, hipertensa, em uso ...",B,"1 Uma mulher com 25 anos, solteira, sem comorb...",A
1,2.\tUm homem com 65 anos de idade encontra-se ...,B,2.\tUma mulher com 23 anos de idade comparece ...,NULA,2.\tUma mulher com 35 anos de idade exerce pro...,C,2.\tAo atender uma primigesta com 15 anos de i...,A,2.\tUm recém-nascido com 20 dias de vida dá en...,C,"2.\tUm paciente com 72 anos de idade, tabagist...",C,"2.\tHomem de 75 anos de idade, tabagista 1 maç...",A,"2.\tUm homem com 62 anos comparece, acompanhad...",D,2.\tUm paciente de 48 anos é internado devido ...,D,"2 Um paciente com 28 anos, vítima de atropelam...",A
2,3.\tUma mulher com 38 anos de idade procurou a...,C,3.\tUm homem com 65 anos de idade encontra-se ...,B,"3.\tUma mulher com 43 anos de idade, com 80 kg...",C,3.\tUma mulher com 35 anos de idade exerce pro...,C,"3.\t Um homem, de 37 anos de idade, com AIDS/H...",B,"3.\tUm menino com 5 anos de idade, acompanhado...",C,3.\tLactente de 6 meses está sendo atendido na...,B,"3.\tUma lactente com 6 meses, nascida a termo,...",B,"3.\tUm menino de 9 anos, com peso de 29 kg e a...",D,3 Um lactente com 3 meses comparece a uma cons...,C
3,4.\tIMAGEM,IMAGEM,"4.\tUma mulher com 32 anos de idade, no quinto...",B,4.\tUma mulher com 18 anos de idade vai ao amb...,A,4.\tUma mulher com 48 anos de idade vai a uma ...,D,4.\tPaciente teve parto cesárea há 8 dias e re...,D,4.\tSegundo o Instituto Nacional do Câncer (IN...,D,"4.\tAdolescente de 14 anos de idade, primípara...",D,"4.\tUma paciente com 21 anos, nuligesta, compa...",A,4.\tUma mulher de 49 anos comparece à unidade ...,B,"4 Uma gestante com 24 anos, G2 P1 A0, chega à ...",D
4,"5.\tUma criança com 4 anos de idade, cujos pai...",A,5.\tIMAGEM,B,5.\tUma criança com 6 anos de idade é levada p...,B,5.\tUma mulher com 18 anos de idade vai ao amb...,A,"5.\tPaciente de 68 anos de idade, do sexo masc...",B,"5.\tUm homem com 48 anos de idade é obeso, tab...",A,5.\tUma mulher de 34 anos com diagnóstico de d...,D,5.\tSindemia é um conjunto de problemas de saú...,A,5.\tEm um território com população em situação...,B,5 Ao assumir a coordenação de uma equipe de sa...,A


## **Installation**

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

## **Unsloth**

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    # Can select any from the below:
    # "unsloth/Qwen2.5-0.5B", "unsloth/Qwen2.5-1.5B", "unsloth/Qwen2.5-3B"
    # "unsloth/Qwen2.5-14B",  "unsloth/Qwen2.5-32B",  "unsloth/Qwen2.5-72B",
    # And also all Instruct versions and Math. Coding verisons!
    model_name = "unsloth/Qwen2.5-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.1: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/106k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.4.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
## **Data Prep**
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Alpaca.ipynb)

For text completions like novel writing, try this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb).

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

<a name="Train"></a>
### **Train the model**
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/51760 [00:00<?, ? examples/s]

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
7.266 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176/7,000,000,000 (0.58% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.959800
2,0.996700
3,1.051400
4,1.136900
5,1.072700
6,0.998300
7,0.740900
8,1.014400
9,0.957900
10,0.916300


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

416.4684 seconds used for training.
6.94 minutes used for training.
Peak reserved memory = 9.379 GB.
Peak reserved memory for training = 2.113 GB.
Peak reserved memory % of max memory = 63.625 %.
Peak reserved memory for training % of max memory = 14.334 %.


<a name="Save"></a>
### **Saving, loading finetuned models**
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

## **Respondendo as questões do Revalida**

In [ ]:
from transformers import TextStreamer

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [ ]:
respostas_llm = {}
for name in sheet_names:
  perguntas = provas['Perguntas-' + name].to_list()
  gabaritos = provas['Gabarito-' + name].to_list()
  respostas_llm['Repostas-' + name] = []

  for pergunta, gabarito in zip(perguntas, gabaritos):

    if gabarito != "IMAGEM" and gabarito != "TABELA" and gabarito != "NULA":

      prompt = f"Dado a questão a seguir, qual item a, b, c ou d está correto? Pense passo a passo"

      inputs = tokenizer(
      [
          alpaca_prompt.format(
              prompt, # instruction
              pergunta, # input
              "", # output - leave this blank for generation!
          )
      ], return_tensors = "pt").to("cuda")

      output = model.generate(**inputs, max_new_tokens=1024)
      resposta_ids = output[0]
      resposta_texto = tokenizer.decode(resposta_ids, skip_special_tokens=True)
      respostas_llm['Repostas-' + name].append(resposta_texto)

    else:
      respostas_llm['Repostas-' + name].append(gabarito)

In [ ]:
from google.colab import files

df = pd.DataFrame(respostas_llm)
df.to_excel('respostas.xlsx', index=False)

# Especificar o caminho do arquivo no Google Drive
caminho_arquivo = '/content/drive/My Drive/respostas.xlsx'

df.to_excel(caminho_arquivo, index=False)

files.download('respostas.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>